# Ray RLlib Multi-Armed Bandits - Market Bandit Example

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

Load a dataset derived from [NYU Stern](http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html) that shows returns for nearly a century of market data, including dividends and adjusted for inflation.

In [107]:
import pandas as pd

def load_market_data (file_name):
    with open(file_name, "r") as f:
        return pd.read_table(f)

In [108]:
df = load_market_data("market.tsv")
df.head()

,year,inflation,sp500,t.bill,t.bond,corp
0,1928,-1.15,45.49,4.28,2.01,4.42
1,1929,0.00,-8.30,3.16,4.20,3.02
2,1930,-2.67,-23.07,7.42,7.41,3.30
3,1931,-8.93,-38.33,12.34,7.00,-7.41
4,1932,-10.30,1.85,12.68,21.28,37.78


In [109]:
df.describe()

,year,inflation,sp500,t.bill,t.bond,corp
count,92.00000,92.000000,92.000000,92.000000,92.000000,92.000000
mean,1973.50000,3.041957,8.413261,0.434239,2.166413,4.216630
std,26.70206,3.803579,19.619605,3.573035,8.126432,8.625809
min,1928.00000,-10.300000,-38.900000,-12.050000,-14.570000,-14.850000
25%,1950.75000,1.415000,-2.740000,-1.185000,-2.620000,-1.322500
50%,1973.50000,2.750000,10.515000,0.590000,1.070000,3.910000
75%,1996.25000,4.275000,20.622500,2.117500,7.037500,9.287500
max,2019.00000,14.390000,58.200000,12.680000,25.140000,37.780000


In [114]:
import gym
from gym.spaces import Discrete, Box
import numpy as np
import random

class MarketBandit (gym.Env):
    MAX_INFLATION = 100.
    TICKERS = ["sp500", "t.bill", "t.bond", "corp"]

    ## NB: using a hard-coded full path here -
    ## there does not appear to be any way to pass any kind of 
    ## configuration into the Tune trials of a bandit?
    FILE = "/Users/paco/src/rllib_tutorials/bandits/market.tsv"
    
    def __init__ (self, config=None):
        print("CONFIG", config)

        self.action_space = Discrete(4)
        self.observation_space = Box(
            low=-self.MAX_INFLATION,
            high=self.MAX_INFLATION,
            shape=(1, )
        )
        self.df = load_market_data(self.FILE)
        self.cur_context = None


    def reset (self):
        self.year = self.df["year"].min()
        #self.cur_context = self.df.loc[df["year"] == self.year]["inflation"]
        self.cur_context = 0
        self.done = False
        self.info = {}

        return [self.cur_context]


    def step (self, action):
        if self.done:
            reward = 0.
            regret = 0.
        else:
            row = self.df.loc[df["year"] == self.year]

            # calculate reward
            ticker = self.TICKERS[action]
            reward = float(row[ticker])

            # calculate regret
            max_reward = max(map(lambda t: float(row[t]), self.TICKERS))
            regret = round(max_reward - reward)

            # update the context
            ## NB: Tune/RLlib throw exceptions about the context 
            ## being out of range for an observation value (it's not)
            #self.cur_context = float(row["inflation"])
            self.cur_context = 0

            # increment the year
            self.year += 1

            if self.year >= self.df["year"].max():
                self.done = True

        context = [self.cur_context]

        self.info = {
            "regret": regret
        }
         
        return [context, reward, self.done, self.info]


    def seed (self, seed=None):
        """Sets the seed for this env's random number generator(s).
        Note:
            Some environments use multiple pseudorandom number generators.
            We want to capture all such seeds used in order to ensure that
            there aren't accidental correlations between multiple generators.
        Returns:
            list<bigint>: Returns the list of seeds used in this env's random
              number generators. The first value in the list should be the
              "main" seed, or the value which a reproducer should pass to
              'seed'. Often, the main seed equals the provided 'seed', but
              this won't be true if seed=None, for example.
        """
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

In [115]:
bandit = MarketBandit()
bandit.reset()

for i in range(10):
    action = bandit.action_space.sample()
    obs = bandit.step(action)
    print(action, obs)

CONFIG None
0 [[0], 45, False, {'regret': 0}]
0 [[0], -8, False, {'regret': 12}]
1 [[0], 7, False, {'regret': 0}]
3 [[0], -7, False, {'regret': 20}]
0 [[0], 2, False, {'regret': 36}]
1 [[0], 6, False, {'regret': 52}]
0 [[0], -5, False, {'regret': 19}]
0 [[0], 43, False, {'regret': 0}]
3 [[0], 10, False, {'regret': 20}]
3 [[0], -8, False, {'regret': 6}]


In [116]:
stop = {
    "training_iteration": 100,
    "timesteps_total": 100000,
    "episode_reward_mean": 20.0,
}

config = {
    "env": MarketBandit,
}

In [117]:
from ray import tune

analysis = tune.run("contrib/LinUCB", config=config, stop=stop)

Trial name,status,loc
contrib_LinUCB_MarketBandit_00000,RUNNING,


(pid=53962) CONFIG {}
(pid=53962) 2020-05-26 14:43:57,706	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
(pid=53962) 2020-05-26 14:43:57,710	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=53962) /opt/anaconda3/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=53962)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=53962) 2020-05-26 14:43:57,723	INFO trainable.py:217 -- Getting current IP.
(pid=53962) 2020-05-26 14:43:57,723	WARNING util.py:37 -- Install gputil for GPU system monitoring.


2020-05-26 14:43:58,258	ERROR trial_runner.py:519 -- Trial contrib_LinUCB_MarketBandit_00000: Error processing event.
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 467, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 431, in fetch_result
    result = ray.get(trial_future[0], DEFAULT_GET_TIMEOUT)
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/worker.py", line 1515, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(RuntimeError): ray::LinUCB.train() (pid=53962, ip=192.168.1.65)
  File "python/ray/_raylet.pyx", line 463, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 417, in ray._raylet.execute_task.function_executor
  File "/opt/anaconda3/lib/python3.7/site-packages/ray/rllib/agents/trainer.py", line 498, in train
    raise e
  File "/opt/anaconda3/lib/pytho

Trial name,status,loc
contrib_LinUCB_MarketBandit_00000,ERROR,
Trial name,# failures,error file
contrib_LinUCB_MarketBandit_00000,1,/Users/paco/ray_results/contrib/LinUCB/contrib_LinUCB_MarketBandit_0_2020-05-26_14-43-51ze06e_dp/error.txt


Trial name,status,loc
contrib_LinUCB_MarketBandit_00000,ERROR,
Trial name,# failures,error file
contrib_LinUCB_MarketBandit_00000,1,/Users/paco/ray_results/contrib/LinUCB/contrib_LinUCB_MarketBandit_0_2020-05-26_14-43-51ze06e_dp/error.txt


TuneError: ('Trials did not complete', [contrib_LinUCB_MarketBandit_00000])